In [28]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [29]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import requests

In [30]:
from ddgs import DDGS

@tool
def search(query: str) -> str:
    """
    Search the web for information using duckduckgo
    """
    search_tool = DDGS()
    results = search_tool.text(query)
    return str(results)

In [ ]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=enter_your_access_key&query={city}'

  response = requests.get(url)

  return response.json()

In [32]:
llm = ChatGroq(model="llama3-8b-8192")

In [33]:
llm.invoke('hi')

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.029330463, 'prompt_time': 0.006204232, 'queue_time': 0.6881101350000001, 'total_time': 0.035534695}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_24ec19897b', 'finish_reason': 'stop', 'logprobs': None}, id='run--38442c5b-ab14-4a8d-b847-9d0894852c3b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [34]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain  import hub

In [35]:
# step 2: Pull the ReAct propt from LangChain Hub

prompt = hub.pull("hwchase17/react") # pulls the standard ReAct agent prompt

c:\Users\91827\Desktop\agents_in_langchain\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [36]:
# step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search, get_weather_data],
    prompt=prompt
)

In [37]:
# step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search, get_weather_data],
    verbose=True
)

In [38]:
# step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})



> Entering new AgentExecutor chain...
Thought: I need to find the capital of Madhya Pradesh first. Then, I can use that information to get the current weather data.

Action: search
Action Input: "capital of Madhya Pradesh"[{'title': 'Madhya Pradesh | History, Government, Map, Capital, & Facts', 'href': 'https://www.britannica.com/place/Madhya-Pradesh', 'body': ''}, {'title': 'Capital of Madhya Pradesh: City, Map & Key Facts Explained', 'href': 'https://www.vedantu.com/geography/capital-of-madhya-pradesh', 'body': 'Find out the capital of Madhya Pradesh, its location, history, and why Bhopal is the state’s political and cultural center.'}, {'title': "Bhopal - BYJU'S", 'href': 'https://byjus.com/social-science/capital-of-madhya-pradesh/', 'body': 'Soon after independence, Nagpur was the capital of Madhya Pradesh. After the States Reorganisation Act was passed, the states of India were divided based on language and …'}, {'title': 'About District | District Bhopal, Government of Madhya P

In [39]:
response['output']

'The capital of Madhya Pradesh is Bhopal, and its current weather condition is hazy with a temperature of 27°C, humidity of 94%, and wind speed of 16 km/h.'